In [1]:
!pip install --upgrade pip
!pip3 install --upgrade pip
!pip install tifffile
!pip install tifffile --user
!pip3 install tifffile
!pip3 install tifffile --user
!sudo python3 -m pip install tifffile
import sys
sys.path.append('/home/ec2-user/.local/bin')

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages (19.1.1)
Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages (19.1.1)
You are using pip version 19.0.2, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%%time
import sagemaker
from sagemaker import get_execution_role
from IPython.core.debugger import set_trace
role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::102554356212:role/service-role/AmazonSageMaker-ExecutionRole-20181129T100657
CPU times: user 948 ms, sys: 124 ms, total: 1.07 s
Wall time: 1.26 s


In [3]:
bucket = sess.default_bucket()  
prefix = 'fresh_train_trial'
print(bucket)

sagemaker-eu-west-1-102554356212


In [4]:
from sagemaker.amazon.amazon_estimator import get_image_uri
training_image = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version="latest")
print (training_image)

685385470294.dkr.ecr.eu-west-1.amazonaws.com/semantic-segmentation:latest


### Setup data

In [5]:
import os
import shutil

# Create directory structure mimicing the s3 bucket where data is to be dumped.
lf8 = '/home/ec2-user/SageMaker/raw_data/lior_figure_eight'
os.makedirs(os.path.join(lf8,'train'), exist_ok=True)
os.makedirs(os.path.join(lf8,'validation'), exist_ok=True)
os.makedirs(os.path.join(lf8,'train_annotation'), exist_ok=True)
os.makedirs(os.path.join(lf8,'validation_annotation'), exist_ok=True)

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'
train_annotation_channel = prefix + '/train_annotation'
validation_annotation_channel = prefix + '/validation_annotation'

In [6]:
%%capture
import boto3
import numpy as np
from skimage import util 
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.io import imread as pngread
from skimage.io import imsave as pngsave
from tifffile import imsave,imread
# gcspath = "gs://segproj/PythonScripts/train/"


# Copy tif from s3 to local sagemaker
files = []
s3 = boto3.resource('s3')
s3_resource = boto3.resource('s3')
s3meadata = s3_resource.Bucket(name='meadata')
num_training_samples = 0
for obj in s3meadata.objects.all():
    if 'tif' in obj.key:
        num_training_samples = num_training_samples+1
        file = obj.key
        if 'segproj/training_data/train/' in obj.key:
            s3.meta.client.download_file('meadata', file, '/tmp/raw.tif')
            inverted_img = util.invert(imread('/tmp/raw.tif'))
            jpgpath = '/tmp/'+file.split('/')[-2]+'_'+'raw.jpg'
            image = exposure.rescale_intensity(inverted_img, out_range=(0, 2**31 - 1))
            imsave(jpgpath,img_as_ubyte(image))
#             set_trace()
            sess.upload_data(path=jpgpath, bucket=bucket, key_prefix=train_channel)
        elif 'segproj/training_data/val/' in obj.key:
            s3.meta.client.download_file('meadata', file, '/tmp/raw.tif')
            inverted_img = util.invert(imread('/tmp/raw.tif'))
            jpgpath = '/tmp/'+file.split('/')[-2]+'_'+'raw.jpg'
            image = exposure.rescale_intensity(inverted_img, out_range=(0, 2**31 - 1))
            imsave(jpgpath,img_as_ubyte(image))
            sess.upload_data(path=jpgpath, bucket=bucket, key_prefix=validation_channel)
    elif 'instances_ids.png' in obj.key:
        file = obj.key
        if 'segproj/training_data/train/' in obj.key:
            s3.meta.client.download_file('meadata', file, '/tmp/instances_ids.png')
            pngpath = '/tmp/'+file.split('/')[-2]+'_'+'raw.png'
            im = pngread('/tmp/instances_ids.png')
            im[im > 0] = 1  #otherwise the trainer will think each cell number is a seperate category
#             im[im==0]=255
            pngsave(pngpath,np.uint8(im), check_contrast=False)
            sess.upload_data(path=pngpath, bucket=bucket, key_prefix=train_annotation_channel)
        elif 'segproj/training_data/val/' in obj.key:
            s3.meta.client.download_file('meadata', file, '/tmp/instances_ids.png')
            pngpath = '/tmp/'+file.split('/')[-2]+'_'+'raw.png'
            im = pngread('/tmp/instances_ids.png')
            im[im > 0] = 1  #otherwise the trainer will think each cell number is a seperate category
#             im[im==0]=255
            pngsave(pngpath,np.uint8(im), check_contrast=False)
            sess.upload_data(path=pngpath, bucket=bucket, key_prefix=validation_annotation_channel)
print(num_training_samples)

In [ ]:
# import matplotlib.pyplot as plt
# from skimage.util import img_as_ubyte
# from skimage import exposure
# image = exposure.rescale_intensity(inverted_img, out_range=(0, 2**31 - 1))
# plt.imshow(img_as_ubyte(im))
# plt.show()

Let us now move our prepared datset to the S3 bucket that we decided to use in this notebook earlier. Notice the following directory structure that is used.

```bash
root 
|-train/
|-train_annotation/
|-validation/
|-validation_annotation/

```
Notice also that all the images in the `_annotation` directory are all indexed PNG files. This implies that the metadata (color mapping modes) of the files contain information on how to map the indices to colors and vice versa. Having an indexed PNG is an advantage as the images will be rendered by image viewers as color images, but the image themsevels only contain integers. The integers are also within `[0, 1 ... c-1, 255]`  for a `c` class segmentation problem, with `255` as 'hole' or 'ignore' class. We allow any mode that is a [recognized standard](https://pillow.readthedocs.io/en/3.0.x/handbook/concepts.html#concept-modes) as long as they are read as integers.

While we recommend the format with default color mapping modes such as PASCAL, we also allow the customers to specify their own label maps. Refer to the [documentation](Permalink-to-label-map-documentation-section) for more details. The label map for the PASCAL VOC dataset, is the default (which we use incase no label maps are provided): 
```json
{
    "scale": 1
}```
This essentially tells us to simply use the images as read as integers as labels directly. Since we are using PASCAL dataset, let us create (recreate the default just for demonstration) label map for training channel and let the algorithm use the default (which is exactly the same for the validation channel). If `label_map` is used, please pass it to the label_map channel.


In [7]:
import json
label_map = { "scale": 1 }
with open('train_label_map.json', 'w') as lm_fname:
    json.dump(label_map, lm_fname)

In [8]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
print(s3_output_location)

s3://sagemaker-eu-west-1-102554356212/fresh_train_trial/output


## Training
Now that we are done with all the setup that is needed, we are ready to train our segmentation algorithm. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job. Let us name our training job as `ss-notebook-demo`. Let us also use a nice-and-fast GPU instance (`ml.p3.2xlarge`) to train.

In [47]:
# Create the sagemaker estimator object.
ss_model = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count = 1, 
                                         train_instance_type = 'ml.p3.16xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         output_path = s3_output_location,
                                         base_job_name = 'fresh-train-trial',
                                         sagemaker_session = sess)

In [48]:
# Setup hyperparameters 
ss_model.set_hyperparameters(backbone='resnet-50', # This is the encoder. Other option is resnet-50
                             algorithm='deeplab', # This is the decoder. Other option is 'psp' and 'deeplab'                             
                             use_pretrained_model='False', # Use the pre-trained model.
                             crop_size=240, # Size of image random crop.                             
                             num_classes=2, # Pascal has 21 classes. This is a mandatory parameter.
                             epochs=500, # Number of epochs to run.
                             learning_rate=0.0001,                             
                             optimizer='rmsprop', # Other options include 'adam', 'rmsprop', 'nag', 'adagrad'.
                             lr_scheduler='poly', # Other options include 'cosine' and 'step'.                           
                             mini_batch_size=2, # Setup some mini batch size.
                             validation_mini_batch_size=2,
                             early_stopping=True, # Turn on early stopping. If OFF, other early stopping parameters are ignored.
                             early_stopping_patience=2, # Tolerate these many epochs if the mIoU doens't increase.
                             early_stopping_min_epochs=10, # No matter what, run these many number of epochs.                             
                             num_training_samples=num_training_samples) # This is a mandatory parameter, 1464 in this case.

Now that the hyperparameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm uses to train.

In [49]:
# Create full bucket names
s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)
s3_train_annotation = 's3://{}/{}'.format(bucket, train_annotation_channel)
s3_validation_annotation = 's3://{}/{}'.format(bucket, validation_annotation_channel)

distribution = 'FullyReplicated'
# Create sagemaker s3_input objects
train_data = sagemaker.session.s3_input(s3_train_data, distribution=distribution, 
                                        content_type='image/jpeg', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution=distribution, 
                                        content_type='image/jpeg', s3_data_type='S3Prefix')
train_annotation = sagemaker.session.s3_input(s3_train_annotation, distribution=distribution, 
                                        content_type='image/png', s3_data_type='S3Prefix')
validation_annotation = sagemaker.session.s3_input(s3_validation_annotation, distribution=distribution, 
                                        content_type='image/png', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 
                 'validation': validation_data,
                 'train_annotation': train_annotation, 
                 'validation_annotation':validation_annotation}

In [ ]:
%%capture
ss_model.fit(inputs=data_channels, logs=True)

In [ ]:
ss_predictor = ss_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

## Inference

Let us convert the image to bytearray before we supply it to our endpoint.

In [ ]:
import matplotlib.pyplot as plt
import PIL

filename = "/home/ec2-user/SageMaker/images/190128_LD2_cleaned_ver1.tif"
# filename = "/home/ec2-user/SageMaker/images/set10_raw.jpg"
im = imread(filename)
num = int(''.join(filter(str.isdigit, str(im.dtype)))) - 1
im = img_as_ubyte(exposure.rescale_intensity(im, out_range=(0, 2**num - 1)))
im = img_as_ubyte(exposure.rescale_intensity(im, out_range=(0, 2**num - 1)))
pngsave(filename.replace('tif','jpg'), im)

%matplotlib inline
plt.imshow(im)
plt.axis('off')
with open(filename, 'rb') as image:
    img = image.read()
    img = bytearray(img)

In [ ]:
%%time 
ss_predictor.content_type = 'image/jpeg'
ss_predictor.accept = 'image/png'
return_img = ss_predictor.predict(img)

Let us display the segmentation mask.

In [ ]:
from PIL import Image
import numpy as np
import io

num_classes = 2
mask = np.array(Image.open(io.BytesIO(return_img)))
plt.imshow(mask)
plt.show()

## Delete the Endpoint
Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [ ]:
sagemaker.Session().delete_endpoint(ss_predictor.endpoint)